In [1]:
!apt-get update
!apt-get install -y wget unzip xvfb libxi6 libgconf-2-4

!wget -q https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.49/linux64/chrome-linux64.zip
!wget -q https://storage.googleapis.com/chrome-for-testing-public/136.0.7103.49/linux64/chromedriver-linux64.zip

!unzip -q chrome-linux64.zip
!unzip -q chromedriver-linux64.zip

!mv chrome-linux64/chrome /usr/bin/google-chrome
!mv chromedriver-linux64/chromedriver /usr/bin/chromedriver

!chmod +x /usr/bin/google-chrome
!chmod +x /usr/bin/chromedriver

!pip install selenium


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,659 kB]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:12 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [77.5 kB]
Get:13 https://r2u.stat.illinois.edu/ubuntu 

In [3]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()

Saving archive.zip to archive.zip


In [4]:
with zipfile.ZipFile("archive.zip", 'r') as zip_ref:
    zip_ref.extractall(".")

# See what folder was extracted
print("Extracted folders/files:", os.listdir("."))

Extracted folders/files: ['.config', 'chromedriver-linux64', 'eminem_lyrics', 'chrome-linux64.zip', 'archive.zip', 'chrome-linux64', 'chromedriver-linux64.zip', 'sample_data']


In [5]:
os.listdir("eminem_lyrics")[:5]

['eminem_eviltwin.txt',
 'eminem_goingthroughchanges.txt',
 'eminem_sofar.txt',
 'eminem_mosh.txt',
 'eminem_river.txt']

In [6]:
lyrics_dir = "eminem_lyrics"
output_file = "eminem_corpus.txt"

nonempty_count = 0

with open(output_file, "w", encoding="utf-8") as out_f:
    for root, _, files in os.walk(lyrics_dir):
        for filename in files:
            if filename.endswith(".txt"):
                filepath = os.path.join(root, filename)
                try:
                    with open(filepath, "r", encoding="utf-8", errors="ignore") as f:
                        content = f.read().strip()
                        if len(content) > 20:
                            out_f.write(content + "\n\n")
                            nonempty_count += 1
                except Exception as e:
                    print(f"Error reading {filepath}: {e}")

print(f"Merged {nonempty_count} songs into {output_file}")


Merged 224 songs into eminem_corpus.txt


In [7]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [9]:
from transformers import GPT2Tokenizer
from datasets import load_dataset

# Load dataset
dataset = load_dataset("text", data_files={"train": "eminem_corpus.txt"})

# Load tokenizer
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token  # GPT2 doesn't have a pad_token

# Tokenize text (no truncation or padding)
def tokenize_function(example):
    return tokenizer(example["text"])

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["text"])


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/47704 [00:00<?, ? examples/s]

In [10]:
block_size = 128

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = (len(concatenated["input_ids"]) // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_dataset = tokenized_dataset.map(group_texts, batched=True)


Map:   0%|          | 0/47704 [00:00<?, ? examples/s]

In [11]:
from transformers import GPT2LMHeadModel, TrainingArguments, Trainer

model = GPT2LMHeadModel.from_pretrained("gpt2")

training_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/eminem/checkpoints",   # Save to Drive
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=2,
    save_steps=500,
    save_total_limit=2,
    logging_steps=100,
    prediction_loss_only=True,
    report_to="wandb",     # optional: logs to wandb
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"]
)


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [12]:
trainer.train()


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: elisensoy (elisensoy-university-of-michigan) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
100,4.338300
200,4.241000
300,4.082100
400,4.046100
500,4.035000
600,3.976200
700,3.909600
800,3.984700
900,3.926100
1000,3.875800


TrainOutput(global_step=5496, training_loss=3.4216889516061397, metrics={'train_runtime': 31066.9553, 'train_samples_per_second': 0.354, 'train_steps_per_second': 0.177, 'total_flos': 717834534912000.0, 'train_loss': 3.4216889516061397, 'epoch': 3.0})

In [14]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("/content/drive/MyDrive/eminem/checkpoints/checkpoint-5496")
model.push_to_hub("Elida-Sensoy/gpt2-eminem-lyrics")



model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Elida-Sensoy/gpt2-eminem-lyrics/commit/c9edd9619acb9e7f521f18f0635bc02c771840ea', commit_message='Upload model', commit_description='', oid='c9edd9619acb9e7f521f18f0635bc02c771840ea', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Elida-Sensoy/gpt2-eminem-lyrics', endpoint='https://huggingface.co', repo_type='model', repo_id='Elida-Sensoy/gpt2-eminem-lyrics'), pr_revision=None, pr_num=None)

In [15]:
tokenizer.push_to_hub("Elida-Sensoy/gpt2-eminem-lyrics")


README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Elida-Sensoy/gpt2-eminem-lyrics/commit/facdaafcdac9e404bdbde87521ed3da408a9568c', commit_message='Upload tokenizer', commit_description='', oid='facdaafcdac9e404bdbde87521ed3da408a9568c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Elida-Sensoy/gpt2-eminem-lyrics', endpoint='https://huggingface.co', repo_type='model', repo_id='Elida-Sensoy/gpt2-eminem-lyrics'), pr_revision=None, pr_num=None)

In [16]:
from transformers import GPT2LMHeadModel

model = GPT2LMHeadModel.from_pretrained("Elida-Sensoy/gpt2-eminem-lyrics")


config.json:   0%|          | 0.00/880 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [17]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel

# Load your custom model and tokenizer
model = GPT2LMHeadModel.from_pretrained("Elida-Sensoy/gpt2-eminem-lyrics")
tokenizer = GPT2Tokenizer.from_pretrained("Elida-Sensoy/gpt2-eminem-lyrics")

# Ensure tokenizer uses pad_token if missing
tokenizer.pad_token = tokenizer.eos_token


tokenizer_config.json:   0%|          | 0.00/556 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

In [31]:

from transformers import pipeline, set_seed

generator = pipeline("text-generation", model="Elida-Sensoy/gpt2-eminem-lyrics")
set_seed(52)  # for reproducibility

prompt = "This is a madhouse."  # your starting line

results = generator(
    prompt,
    max_length=200,
    num_return_sequences=3,      #  generate 3 completions
    temperature=1.85,             # higher = more creative
    top_p=0.95,                  #  nucleus sampling
    do_sample=True,
    truncation=True,
    pad_token_id=50256           # prevents warnings with GPT-2
)

for i, res in enumerate(results):
    print(f"\n🎵 Verse {i+1}:\n{res['generated_text']}")


Device set to use cpu



🎵 Verse 1:
This is a madhouse. If anybody wants a little elbow roomThe game's on.They say I love you too, so go and fuck an Elco-owned beat!Oh and try the shit I do here; all-you-fucking kids just keep getting olderAnd they want nothing more.There's more that they want, that they do not want at allThe new kid on the block, who's as tough to beThe first word to everyone's list when you say the exact same thingBut everyone has something interesting saidNow take this chanceYou just sent the whole Hip Hop Department into convulsionsYou'll end up having your whole school run rioted in reactionYou will be sued 'round'The whole country's just gonna gather at willA little tear-filled room full of thugs to stage an orderly assaultIf any one's watching them nowYou can all watch 'cause it might come back out the gate in what comes nextThe sight does nothing for nothing is esc

🎵 Verse 2:
This is a madhouse.So what's the problem bitch, a madhouse just madas me??The problem is the other rapper jus